<a href="https://colab.research.google.com/github/szperan21/BasicPythonMiniProject/blob/main/NumpyMatplotlibMiniProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install mpi4py

  Using cached mpi4py-4.0.3.tar.gz (466 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-4.0.3-cp311-cp311-linux_x86_64.whl size=4458266 sha256=e2c116a93ff374292d2c8b1d9357bcbbfe9f3f2db2e202b29d2f2472bd1eb294
  Stored in directory: /root/.cache/pip/wheels/5c/56/17/bf6ba37aa971a191a8b9eaa188bf5ec855b8911c1c56fb1f84
Successfully built mpi4py


In [6]:
import numpy as np
import sys
import datetime
import math
from mpi4py import MPI

In [7]:
def inside_circle(total_count):
    x = np.random.uniform(size=total_count)
    y = np.random.uniform(size=total_count)
    radii = np.sqrt(x*x + y*y)
    count = len(radii[np.where(radii<=1.0)])
    return count


In [ ]:
#initialize vars for MPI
comm = MPI.COMM_WORLD
cpus = comm.Get_size()
rank = comm.Get_rank()

PiSimData = np.array([])

for loopControl in range(2,10):
  n_samples = 10**loopControl

  if rank == 0:
    start_time = datetime.datetime.now()
    partitions = [ int(n_samples / cpus) ] * cpus
    counts = [ int(0) ] * cpus
  else:
    partitions = None
    counts = None

  partition_item = comm.scatter(partitions, root=0)
  count_item = inside_circle(partition_item)
  counts = comm.gather(count_item, root=0)

  if rank == 0:
    #Record results of the simulation
    my_pi = 4.0 * sum(counts) / sum(partitions)
    size_of_float = np.dtype(np.float64).itemsize
    memory_required = 3 * n_samples * size_of_float / (1024**3)
    end_time = datetime.datetime.now()
    elapsed_time = (end_time - start_time).total_seconds()
    relative_error = abs(math.pi-my_pi)/math.pi

    #append to PiSimData
    PiSimData = PiSimData.append(my_pi)
    PiSimData = PiSimData.append(size_of_float)
    PiSimData = PiSimData.append(memory_required)
    PiSimData = PiSimData.append(end_time)
    PiSimData = PiSimData.append(elapsed_time)
    PiSimData = PiSimData.append(relative_error)

    print("N:", n_samples, " Pi:", my_pi, " Memory:", memory_required, "GiB", " Time:", elapsed_time, "s", "Relative Error:", relative_error)


#Write out the results to a CSV
np.savetxt('PiSimData', PiSimData, delimiter=',')


N: 100  Pi: 2.84  Memory: 2.2351741790771484e-06 GiB  Time: 0.000416 s Relative Error: 0.09599992323803451
N: 1000  Pi: 3.156  Memory: 2.2351741790771484e-05 GiB  Time: 0.000693 s Relative Error: 0.004586000796043443
N: 10000  Pi: 3.1204  Memory: 0.00022351741790771484 GiB  Time: 0.00061 s Relative Error: 0.00674583115209953
N: 100000  Pi: 3.14212  Memory: 0.0022351741790771484 GiB  Time: 0.016695 s Relative Error: 0.00016785957581232083
N: 1000000  Pi: 3.14122  Memory: 0.022351741790771484 GiB  Time: 0.069644 s Relative Error: 0.00011861932175298856
N: 10000000  Pi: 3.1421944  Memory: 0.22351741790771484 GiB  Time: 0.48958 s Relative Error: 0.00019154183134451042
N: 100000000  Pi: 3.14164144  Memory: 2.2351741790771484 GiB  Time: 5.877195 s Relative Error: 1.5529196680235967e-05
